In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from selenium import webdriver  
from selenium.webdriver.common.by import By

In [3]:
conference = 'cvpr'
year = 2023

conference_url = f"https://openaccess.thecvf.com/CVPR{year}?day=all"
chromedriver_path = '/usr/bin/chromedriver' 
data_path = Path("./papers_info")

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    options=options
)

In [9]:
driver.get(conference_url)

In [32]:
title_element_list = driver.find_elements(By.CLASS_NAME, 'ptitle')

titles = []
absts = []
paper_links = []

for element in title_element_list:
    paper_links.append(element.find_element(By.TAG_NAME, 'a').get_attribute('href'))

for i,element in enumerate(title_element_list):
    driver.get(paper_links[i])
    title = driver.find_element(By.ID, 'papertitle').text
    abst = driver.find_element(By.ID, 'abstract').text
    titles.append(title)
    absts.append(abst)

In [ ]:
driver.quit()

In [ ]:
cvpr = pd.DataFrame.from_dict({
    'year': year,
    'title': titles, 
    'abstract': absts,
})

In [ ]:
mask = np.array([len(a) >= 200 for a in cvpr.abstract])
cvpr = cvpr[mask].reset_index(drop=True)
print(f'Removing {np.sum(~mask)} submissions with abstract length below 200 characters.')

In [ ]:
cvpr.to_pickle(data_path / f'{conference}_{year}.pickle.zip')